In [1]:
# ACCESO A API DE INE FOR MACRODATA

In [2]:
# Libs 
# codigo = "HPT40012" # Hipotec Andalucia
# codigo = "HPT40013" # Total Nacional
# codigo = "T30681" PIB
# codigo = "HPT40012" # Hipotec Andalucia
# codigo = "CNTR3179" # PIB Total Nacional. Variación anual BASE 2010 MACRO_02
# codigo = "CNTR4892" # PIB Total Nacional. Variación anual BASE 2015 MACRO_02
# codigo = "IPC206448" # IPC total Nacional . Variación Anual MACRO_03
# codigo = "CTNFSI6925" # Dato base. Total Nacional. Tasa de Ahorro hogares
# codigo = "EPA88" # Poblac Activa
# codigo = "EPA87" # Poblac Ocupada
# codigo = "EPA87" # Poblac Ocupada var inter-anual MACRO_08
# codigo = "EPA86" # Poblac de parados MACRO_09
# codigo = "EPA815" # Tasa de Desempleo MACRO_10
# codigo ="T28182" # Estructura Salarial.
num_datos = 400

In [3]:
MACRO_DIC={"EPA87":""}
num_datos = 400

In [4]:
# Static lists for purpose of illustration
macro_codes = ["MACR0_01", "MACR0_02", "MACRO_03", 
               "MACRO_06", "MACRO_07", "MACRO_08", 
               "MACRO_09", "MACRO_10"]

ine_codes = ["CNTR3179", "CNTR4892", "IPC206448",
             "EPA88","EPA87","EPA87",
             "EPA86","EPA815"]

macro_dsc_codes = ["PIB", "PIB", "IPC", 
                  "POBL_ACTIVA","POBL_OCUPADA","POBL_OCUPADA_VARINTERA",
                  "POBL_PARADOS","TASA_PARO"]

MACRO_DICT = {}
for i in range(len(macro_codes)):
    MACRO_DICT[macro_codes[i]] = (ine_codes[i],macro_dsc_codes[i])


In [5]:
MACRO_DICT

{'MACR0_01': ('CNTR3179', 'PIB'),
 'MACR0_02': ('CNTR4892', 'PIB'),
 'MACRO_03': ('IPC206448', 'IPC'),
 'MACRO_06': ('EPA88', 'POBL_ACTIVA'),
 'MACRO_07': ('EPA87', 'POBL_OCUPADA'),
 'MACRO_08': ('EPA87', 'POBL_OCUPADA_VARINTERA'),
 'MACRO_09': ('EPA86', 'POBL_PARADOS'),
 'MACRO_10': ('EPA815', 'TASA_PARO')}

In [6]:
PATH="/content/data/INE/"


In [7]:

%ls -l "/content/data/INE/"


total 0


In [8]:
print ("Dict key-value are : ") 
print([(k, MACRO_DICT[k]) for k in MACRO_DICT])

Dict key-value are : 
[('MACR0_01', ('CNTR3179', 'PIB')), ('MACR0_02', ('CNTR4892', 'PIB')), ('MACRO_03', ('IPC206448', 'IPC')), ('MACRO_06', ('EPA88', 'POBL_ACTIVA')), ('MACRO_07', ('EPA87', 'POBL_OCUPADA')), ('MACRO_08', ('EPA87', 'POBL_OCUPADA_VARINTERA')), ('MACRO_09', ('EPA86', 'POBL_PARADOS')), ('MACRO_10', ('EPA815', 'TASA_PARO'))]


In [11]:
!pip install boto3

     |████████████████████████████████| 132 kB 14.6 MB/s 
     |████████████████████████████████| 8.6 MB 59.2 MB/s 
     |████████████████████████████████| 79 kB 8.5 MB/s 
     |████████████████████████████████| 138 kB 61.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.8 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [9]:
import boto3

In [18]:
# pip install requests

     |████████████████████████████████| 127 kB 13.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.8
    Uninstalling urllib3-1.26.8:
      Successfully uninstalled urllib3-1.26.8
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [10]:
import requests

In [11]:
import boto3
REGION='us-east-1'
s3 = boto3.client('s3', region_name=REGION)

In [12]:
# MADAWS
import os
os.environ['AWS_ACCESS_KEY_ID']=''
os.environ['AWS_SECRET_ACCESS_KEY']=''


In [17]:
def get_data_ine_serie(path, macro_codigo, codigo_ine, num_datos):
    """
    This function get data INE MACRODATA
    """
    url_plantilla = 'http://servicios.ine.es/wstempus/js/ES/DATOS_SERIE/{codigo}?nult={num_datos}'

    codigo = codigo_ine
    # num_datos = 100

    url = url_plantilla.format(codigo=codigo,
                               num_datos=num_datos)

    respuesta = requests.get(url)
    datos = respuesta.json()

    nombre = datos['Nombre']

    fecha_serie_utc = pd.to_datetime([x['Fecha'] for x in datos['Data']], unit='ms', utc=True)

    fecha_serie_madrid = fecha_serie_utc.tz_convert('Europe/Madrid')

    fecha_serie = [x.tz_localize(None).date()
                   for x in fecha_serie_madrid]

    ocupados_serie = [x['Valor']
                      for x in datos['Data']] 
   
    

    tabla = pd.DataFrame(ocupados_serie,
                         index=fecha_serie,
                         columns=[macro_codigo])
    
    tabla['COD_M']=macro_codigo
  #  tabla.reset_idex()
    tabla.reset_index().rename(columns={'index': 'FECHA'}).to_csv(f'{path}df_{macro_codigo}.csv', index=False, header=True , sep="|", encoding='utf-8')
    #tabla.reset_index().rename(columns={'index': 'FECHA'}).to_csv(f'PATH/df_{macro_codigo}.csv', index=False, header=True , sep="|", encoding='utf-8')
   # tabla.set_index('fecha_serie')
    # tabla.set_index('fecha_serie')
    # Example Get PIB variación interanual
    # get_data_ine_serie(PATH ,params_codes_list[1][0],params_codes_list[1][1],100)
    # Boto3 para subir ficheros al datalake
    s3 = boto3.client('s3', region_name=REGION)
    BUCKET_NAME = 'mad.bs31'
    filename=f'df_{macro_codigo}.csv'
    s3.upload_file(f'{path}/df_{macro_codigo}.csv', BUCKET_NAME, filename)



In [14]:
print(PATH, params_codes_list[1][0],params_codes_list[1][1],100)

NameError: ignored

In [15]:
import pandas as pd

In [18]:
get_data_ine_serie(PATH ,'MACRO_02','CNTR4892',100)

In [19]:
get_data_ine_serie(PATH ,'MACRO_06','EPA88',100)

In [20]:
# Prob en IPC por formato date 
# get_data_ine_serie(PATH ,'MACRO_03','IPC206448',100)

KeyError: ignored

In [ ]:
s3 = boto3.client('s3', region_name=REGION)

#filename = f'./app/{filename}'
bucket_name = 'mad.bs31'
filename_source=f'./app/df_MACRO_03.csv'
filename_target=f'df_MACRO_03.csv'


s3.upload_file(filename_source, bucket_name, filename_target)

In [28]:
data1 = pd.read_csv(f'{PATH}/df_MACRO_02.csv', sep="|")

In [29]:
data1

,FECHA,MACRO_02,COD_M
0,1997-01-01,3.1804,MACRO_02
1,1997-04-01,3.3534,MACRO_02
2,1997-07-01,3.6944,MACRO_02
3,1997-10-01,4.5662,MACRO_02
4,1998-01-01,4.4223,MACRO_02
...,...,...,...
95,2020-10-01,-8.8016,MACRO_02
96,2021-01-01,-4.2504,MACRO_02
97,2021-04-01,17.6613,MACRO_02
98,2021-07-01,3.3884,MACRO_02


In [ ]:
params_codes_list = [(k, MACRO_DICT[k]) for k in MACRO_DICT]

In [ ]:
params_codes_list[1][1][0]

In [ ]:
tabla.head()

In [ ]:
# get_data_ine_serie(PATH,params_codes_list[1][0],params_codes_list[1][1],100)

In [ ]:
import requests
import pandas as pd

url_plantilla = 'http://servicios.ine.es/wstempus/js/ES/DATOS_SERIE/{codigo}?nult={num_datos}'
# IPC206448
codigo = "EPA87"
num_datos = 100

url = url_plantilla.format(codigo=codigo,
                           num_datos=num_datos)

respuesta = requests.get(url)
datos = respuesta.json()

nombre = datos['Nombre']

fecha_serie_utc = pd.to_datetime([x['Fecha'] for x in datos['Data']], unit='ms', utc=True)

fecha_serie_madrid = fecha_serie_utc.tz_convert('Europe/Madrid')

fecha_serie = [x.tz_localize(None).date()
               for x in fecha_serie_madrid]

ocupados_serie = [x['Valor']
                  for x in datos['Data']] 

tabla = pd.DataFrame(ocupados_serie,
                     index=fecha_serie,
                     columns=[nombre])



In [ ]:
tabla.info()

In [ ]:
tabla.index=pd.to_datetime(tabla.index)

In [ ]:
tabla.loc['2019']

In [ ]:
ax=tabla.plot(marker='o', linestyle='-')

In [ ]:
ax=tabla.loc['2019':'2020'].plot(marker='o', linestyle='-')

In [ ]:
tabla.rolling(window=3).mean()

In [ ]:
tabla_year=tabla
# cases
        increase = reg_df['cases'] - reg_df['cases'].shift(1)
        increase[increase < 0] = 0.0
        rolling = increase.rolling(window=3).mean()
        df['increase_cases'].mask(df.region == region, increase, inplace=True)
        df['rolling_cases'].mask(df.region == region, rolling, inplace=True)
        df['increase_cases_per_100k'] = df['increase_cases'] * 100_000 / df['population']
        df['rolling_cases_per_100k'] = df['rolling_cases'] * 100_000 / df['population']
        # cases acum
        rolling = increase.rolling(window=14).sum()

In [ ]:
https://servicios.ine.es/wstempus/js/ES/DATOS_SERIE/HPT40012?nult=200

In [ ]:
import requests
import pandas as pd

url_plantilla = 'http://servicios.ine.es/wstempus/js/ES/DATOS_SERIE/{codigo}?nult={num_datos}'

codigo = "HPT40012" # Hipotec Andalucia
codigo = "HPT40013" # Total Nacional
num_datos = 400

url = url_plantilla.format(codigo=codigo,
                           num_datos=num_datos)

respuesta = requests.get(url)
datos = respuesta.json()

nombre = datos['Nombre']

fecha_serie_utc = pd.to_datetime([x['Fecha'] for x in datos['Data']], unit='ms', utc=True)

fecha_serie_madrid = fecha_serie_utc.tz_convert('Europe/Madrid')

fecha_serie = [x.tz_localize(None).date()
               for x in fecha_serie_madrid]

ocupados_serie = [x['Valor']
                  for x in datos['Data']] 

tabla = pd.DataFrame(ocupados_serie,
                     index=fecha_serie,
                     columns=[nombre])


In [ ]:
tabla.tail() # 2013-2019

In [ ]:
tabla.rename(columns={'Total. Número de hipotecas. Andalucía. Base nueva. Anual. Total fincas.':'Tot_Hipo'},inplace=True)

In [ ]:
tabla.info()

In [ ]:
tabla.index = pd.to_datetime(tabla.index)

In [ ]:
list(tabla.columns)

In [ ]:
tabla.rename(columns={'Total. Número de hipotecas. Total Nacional. Base nueva. Anual. Total fincas. ':'TotNacHipo'},inplace=True)

In [ ]:
tabla.loc['2003':'2005']

In [ ]:
tabla.idxmax()

In [ ]:
tabla.loc['2006-01-01']

In [ ]:
#tabla_annual = pd.DataFrame()
#tabla_annual['Tot_Hipo_Year'] = 
tabla_year = tabla.resample('AS').sum()

In [ ]:
tabla_year.plot(marker='o', linestyle='-')

In [ ]:
last_date = tabla.index.values

In [ ]:
last_date

In [ ]:
tabla.tail()

In [ ]:
# Resampling

In [24]:
%head -3 "/content/data/INE/df_MACRO_02.csv"

UsageError: Line magic function `%head` not found.
